### Runnable

In [ ]:
from langchain.prompts import PromptTemplate

# 프롬프트를 생성합니다. 여기서는 숫자(num)를 입력 받으면 10배한 값을 출력하는 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{num} 의 10배는?")
prompt

In [ ]:
from langchain_openai import ChatOpenAI

# chain 을 생성합니다.
chain = prompt | ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
# chain 을 실행합니다.
chain.invoke({"num": 5})


### 데이터를 효과적으로 전달하는 방법

In [ ]:
# RunnablePassthrough는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다. 이는 주로 RunnableParallel과 함께 사용되어 맵에서 새 키에 데이터를 할당하는 데 사용됩니다.
# RunnablePassthrough()가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.
# RunnablePassthrough가 assign (RunnablePassthrough.assign(...))과 함께 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

from langchain_core.runnables import RunnablePassthrough


#### RunnablePassthrough

In [ ]:
# chain 을 실행합니다.
chain.invoke({"num": 2})


In [ ]:
# 아래는 RunnablePassthrough 를 사용한 예제입니다.
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경되었습니다.
runnable_chain.invoke(10)


In [ ]:
# 다음은 RunnablePassthrough.assign() 을 사용하는 경우와 비교한 결과입니다.

RunnablePassthrough().invoke({"num": 1})


In [ ]:
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

#### RunnableParallel

In [ ]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스를 생성합니다. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행할 수 있습니다.
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드를 호출합니다.
runnable.invoke({"num": 1})


In [ ]:
# Chain 도 RunnableParallel 적용할 수 있습니다.

chain1 = (
    {"num": RunnablePassthrough()}
    | PromptTemplate.from_template("{num} 의 10배는?\n답변(결과만): ")
    | ChatOpenAI()
)
chain2 = (
    {"num": RunnablePassthrough()}
    | PromptTemplate.from_template("{num} 의 1/10배는?\n답변(결과만): ")
    | ChatOpenAI()
)


In [ ]:
combined_chain = RunnableParallel(a=chain1, b=chain2)
combined_chain.invoke({"num": 10})


#### RunnableLambda

In [ ]:
# RunnableLambda 를 사용하여 사용자 정의 함수를 맵핑할 수 있습니다.

from langchain_core.runnables import RunnableLambda
def extra(x):
    ext = x["extra"]
    return int(ext["num"]) * int(ext["mult"])
(runnable | RunnableLambda(extra)).invoke({"num": 3})